In [1]:
!pip install gensim

  Attempting uninstall: Cython
    Found existing installation: Cython 0.29.24
    Uninstalling Cython-0.29.24:
      Successfully uninstalled Cython-0.29.24


In [2]:
import pandas as pd
import gensim, logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [3]:
data = pd.read_csv('datasets/amazon/new_sampled.csv')

In [11]:
# Building sentences

titles = data['title'].values.tolist()
f = lambda x : x.split(' ')
sequences = list(map(f,titles))

In [12]:
len(sequences)

98828

In [15]:
sequences = list(filter(lambda x: x != '',sequences))

In [77]:
model = gensim.models.Word2Vec(sequences, min_count=2,vector_size=200,epochs=10)

2022-01-12 23:03:50,756 : INFO : collecting all words and their counts
2022-01-12 23:03:50,757 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2022-01-12 23:03:50,779 : INFO : PROGRESS: at sentence #10000, processed 90541 words, keeping 26604 word types
2022-01-12 23:03:50,801 : INFO : PROGRESS: at sentence #20000, processed 181395 words, keeping 42453 word types
2022-01-12 23:03:50,825 : INFO : PROGRESS: at sentence #30000, processed 271550 words, keeping 55493 word types
2022-01-12 23:03:50,848 : INFO : PROGRESS: at sentence #40000, processed 361934 words, keeping 66867 word types
2022-01-12 23:03:50,874 : INFO : PROGRESS: at sentence #50000, processed 452396 words, keeping 77300 word types
2022-01-12 23:03:50,897 : INFO : PROGRESS: at sentence #60000, processed 542759 words, keeping 86764 word types
2022-01-12 23:03:50,926 : INFO : PROGRESS: at sentence #70000, processed 633201 words, keeping 95985 word types
2022-01-12 23:03:50,950 : INFO : PROGRESS: at s

In [78]:
model.wv.most_similar('Dell')

[('cartridge', 0.9744598269462585),
 ('Xerox', 0.9691468477249146),
 ('Canon', 0.9683713316917419),
 ('Brother', 0.9680795669555664),
 ('CLOVER', 0.9669252038002014),
 ('Imaging', 0.9667531251907349),
 ('MPI', 0.9658995866775513),
 ('Cartridge-Black', 0.9601041078567505),
 ('LaserJet', 0.9591062068939209),
 ('ImageClass', 0.958953857421875)]

In [79]:
model.wv.similarity('HP','Dell')

0.9481874

In [88]:
model.wv.similarity('Xerox','Intel')

0.83795893

In [89]:
v = model.wv['HP'] - model.wv['Computer'] + model.wv['Laptop']

In [92]:
model.wv.most_similar(v)

[('HP', 0.9715544581413269),
 ('Dell', 0.947457492351532),
 ('Canon', 0.9356963634490967),
 ('51645A', 0.9339503049850464),
 ('cartridge', 0.9334606528282166),
 ('Brother', 0.9217835664749146),
 ('Axiom', 0.9183753132820129),
 ('Toner', 0.9136387705802917),
 ('Compatible', 0.9089564085006714),
 ('Rhinotek', 0.9088978171348572)]

In [83]:
model.wv.save_word2vec_format('models/embeddings/w2v.txt', binary=False)

2022-01-12 23:05:33,048 : INFO : storing 42467x200 projection weights into models/embeddings/w2v.txt


In [84]:
!python -m spacy init vectors en models/embeddings/w2v.txt models/spacy_embeddings --name title_vectors

[i] Creating blank nlp object for language 'en'

2022-01-12 23:12:11.849124: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'cudart64_110.dll'; dlerror: cudart64_110.dll not found
2022-01-12 23:12:11.849462: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
[2022-01-12 23:12:19,343] [INFO] Reading vectors from models\embeddings\w2v.txt

0it [00:00, ?it/s]
1832it [00:00, 18137.40it/s]
3646it [00:00, 17325.75it/s]
5469it [00:00, 17649.94it/s]



[+] Successfully converted 42467 vectors
[+] Saved nlp object with vectors to output directory. You can now use the path
to it in your config as the 'vectors' setting in [initialize].
C:\Users\souam\Documents\DS\Projects\brand-ner\models\spacy_embeddings


7295it [00:00, 17871.41it/s]
9180it [00:00, 18155.66it/s]
10997it [00:00, 18099.10it/s]
12811it [00:00, 18055.01it/s]
14637it [00:00, 18063.09it/s]
16444it [00:00, 18009.28it/s]
18295it [00:01, 18107.47it/s]
20106it [00:01, 18107.11it/s]
21941it [00:01, 18126.36it/s]
23754it [00:01, 17911.71it/s]
25546it [00:01, 17859.83it/s]
27347it [00:01, 17852.40it/s]
29154it [00:01, 17916.44it/s]
30946it [00:01, 17858.25it/s]
32732it [00:01, 17700.35it/s]
34528it [00:01, 17724.82it/s]
36301it [00:02, 17726.35it/s]
38074it [00:02, 17623.81it/s]
39837it [00:02, 17572.97it/s]
41677it [00:02, 17766.21it/s]
42467it [00:02, 17863.34it/s]
[2022-01-12 23:12:21,726] [INFO] Loaded vectors from models\embeddings\w2v.txt


In [85]:
import spacy
nlp = spacy.load('models/spacy_embeddings')

In [86]:
nlp.add_pipe('ner')
nlp.to_disk('models/spacy_embeddings')

In [87]:
!python -m spacy train models/spacy_embeddings/config.cfg --output models/ner_amazon_embedding --paths.train datasets/amazon/spacy/train_shuffle.spacy --paths.dev datasets/amazon/spacy/dev_shuffle.spacy

^C
